## Read SEG-Y with `obspy`

Before going any further, you might like to know, [What is SEG-Y?](http://www.agilegeoscience.com/blog/2014/3/26/what-is-seg-y.html). See also the articles in [SubSurfWiki](http://www.subsurfwiki.org/wiki/SEG_Y) and [Wikipedia](https://en.wikipedia.org/wiki/SEG_Y).

We'll use the [obspy](https://github.com/obspy/obspy) seismology library to read and write SEGY data.
    
Technical SEG-Y documentation:

* [SEG-Y Rev 1](http://seg.org/Portals/0/SEG/News%20and%20Resources/Technical%20Standards/seg_y_rev1.pdf)
* [SEG-Y Rev 2 proposal](https://www.dropbox.com/s/txrqsfuwo59fjea/SEG-Y%20Rev%202.0%20Draft%20August%202015.pdf?dl=0) and [draft repo](http://community.seg.org/web/technical-standards-committee/documents/-/document_library/view/6062543)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ls -l ../data/*.sgy

## 2D data

In [ ]:
filename = '../data/HUN00-ALT-01_STK.sgy'

In [ ]:
from obspy.io.segy.segy import _read_segy
section = _read_segy(filename)

# OPTIONS
# headonly=True — only reads the header info, then you can index in on-the-fly.
# unpack_headers=True — slows you down here and isn't really required.

In [ ]:
data = np.vstack([t.data for t in section.traces])

In [ ]:
plt.figure(figsize=(16,8))
plt.imshow(data.T, cmap="Greys")
plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
section.traces[0]

In [ ]:
section.textual_file_header

Aargh... 

OK, fine, we'll reformat this.

In [ ]:
def chunk(string, width=80):
    try:
        # Make sure we don't have a ``bytes`` object.
        string = string.decode()
    except:
        # String is already a string, carry on.
        pass
    lines = int(np.ceil(len(string) / width))
    result = ''
    for i in range(lines):
        line = string[i*width:i*width+width]
        result += line + (width-len(line))*' ' + '\n'
    return result

s = section.textual_file_header.decode()
print(chunk(s))

In [ ]:
section.traces[0]

In [ ]:
t = section.traces[0]

t.npts

In [ ]:
t.header

## 3D data

Either use the small volume, or **[get the large dataset from Agile's S3 bucket](https://s3.amazonaws.com/agilegeo/Penobscot_0-1000ms.sgy.gz)**

In [ ]:
#filename = '../data/F3_very_small.sgy'
filename = '../data/Penobscot_0-1000ms.sgy'

In [ ]:
from obspy.io.segy.segy import _read_segy

raw = _read_segy(filename)

In [ ]:
data = np.vstack([t.data for t in raw.traces])

I happen to know that the shape of this dataset is 601 &times; 481.

In [ ]:
_, t = data.shape
seismic = data.reshape((601, 481, t))

Note that we don't actually need to know the last dimension, if we already have two of the three dimensions. `np.reshape()` can compute it for us on the fly:

In [ ]:
seismic = data.reshape((601, 481, -1))

Plot the result...

In [ ]:
clip = np.percentile(seismic, 99)
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.imshow(seismic[100,:,:].T, cmap="Greys", vmin=-clip, vmax=clip)
plt.colorbar(label="Amplitude", shrink=0.8)
ax.set_xlabel("Trace number")
ax.set_ylabel("Time sample")
plt.show()

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2016</p>
</div>